In [5]:
import rasterio
import numpy as np
import rasterio

with rasterio.open('C:/Users/MI18877/Auto-GPT/auto_gpt_workspace/Tesis/scripts/raster_VegUso.tif') as src:
    raster_veguso = src.read(1)

with rasterio.open('C:/Users/MI18877/Auto-GPT/auto_gpt_workspace/Tesis/scripts/raster_DENUE.tif') as src:
    raster_denue = src.read(1)


In [6]:
# Cargar los rasters y leer sus metadatos
with rasterio.open('raster_DENUE.tif') as src_denue:
    denue_meta = src_denue.meta
    denue_bounds = src_denue.bounds
    denue_crs = src_denue.crs

with rasterio.open('raster_VegUso.tif') as src_veguso:
    veguso_meta = src_veguso.meta
    veguso_bounds = src_veguso.bounds
    veguso_crs = src_veguso.crs

# Comparar las propiedades de los rasters
def compare_raster_properties(meta1, meta2, bounds1, bounds2, crs1, crs2):
    print("Tamaño del raster 1:", meta1['width'], "x", meta1['height'])
    print("Tamaño del raster 2:", meta2['width'], "x", meta2['height'])
    
    print("\nCRS del raster 1:", crs1)
    print("CRS del raster 2:", crs2)
    
    print("\nLímites del raster 1 (minx, miny, maxx, maxy):", bounds1)
    print("Límites del raster 2 (minx, miny, maxx, maxy):", bounds2)

compare_raster_properties(denue_meta, veguso_meta, denue_bounds, veguso_bounds, denue_crs, veguso_crs)


Tamaño del raster 1: 1421 x 1723
Tamaño del raster 2: 1486 x 2012

CRS del raster 1: EPSG:32614
CRS del raster 2: EPSG:32614

Límites del raster 1 (minx, miny, maxx, maxy): BoundingBox(left=462765.98571002856, bottom=2113230.1356805954, right=505399.22131148813, top=2164930.99435582)
Límites del raster 2 (minx, miny, maxx, maxy): BoundingBox(left=461655.7433000002, bottom=2106184.9898000006, right=506235.7433000002, top=2166544.9898000006)


In [7]:
intersection_bounds = (
    max(denue_bounds.left, veguso_bounds.left),
    max(denue_bounds.bottom, veguso_bounds.bottom),
    min(denue_bounds.right, veguso_bounds.right),
    min(denue_bounds.top, veguso_bounds.top)
)

In [8]:
from rasterio.mask import mask
from shapely.geometry import box

# Crear un objeto geométrico 'box' con los límites de intersección
intersection_geometry = [box(*intersection_bounds)]

# Función para recortar un raster al área de intersección
def crop_raster_to_intersection(raster_path, intersection_geometry):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, intersection_geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
    return out_image, out_meta

# Recortar ambos rasters al área de intersección
denue_cropped, denue_meta_cropped = crop_raster_to_intersection('raster_DENUE.tif', intersection_geometry)
veguso_cropped, veguso_meta_cropped = crop_raster_to_intersection('raster_VegUso.tif', intersection_geometry)


In [9]:
import numpy as np
from rasterio.warp import reproject, Resampling

# Crear un array vacío con las dimensiones del raster 1
veguso_cropped_resampled = np.empty((1, denue_meta_cropped['height'], denue_meta_cropped['width']), dtype=np.float32)

# Re-muestrear el raster 2 para que coincida con las dimensiones del raster 1
reproject(
    source=veguso_cropped,
    destination=veguso_cropped_resampled,
    src_transform=veguso_meta_cropped['transform'],
    src_crs=veguso_meta_cropped['crs'],
    dst_transform=denue_meta_cropped['transform'],
    dst_crs=denue_meta_cropped['crs'],
    resampling=Resampling.bilinear
)

# Actualizar el metadato de la altura y el ancho del raster 2
veguso_meta_cropped_resampled = veguso_meta_cropped.copy()
veguso_meta_cropped_resampled['height'] = denue_meta_cropped['height']
veguso_meta_cropped_resampled['width'] = denue_meta_cropped['width']

In [10]:
print("Tamaño del raster recortado y re-muestreado 1:", veguso_meta_cropped_resampled['height'], "x", veguso_meta_cropped_resampled['width'])
print("Tamaño del raster recortado 2:", denue_meta_cropped['height'], "x", denue_meta_cropped['width'])

Tamaño del raster recortado y re-muestreado 1: 1723 x 1421
Tamaño del raster recortado 2: 1723 x 1421


In [ ]:
# Definir los pesos para cada fuente de datos
weight_denue = 1
weight_veguso = 2

# Combinar los rasters recortados utilizando los pesos definidos
combined_raster = (veguso_cropped_resampled * weight_veguso + denue_cropped * weight_denue) / (weight_veguso + weight_denue)

# Guardar el raster combinado
output_raster_path = 'raster_combined.tif'
with rasterio.open(output_raster_path, 'w', **denue_meta_cropped) as dst:
    dst.write(combined_raster)

In [37]:
output_raster_path = 'raster_combined.tif'
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio import features
from rasterio.plot import show
# Leer el archivo raster creado previamente
with rasterio.open(output_raster_path) as src:
    saved_raster_data = src.read(1)
    raster_transform = src.transform

# Visualizar el archivo rasterizado
fig, ax = plt.subplots(figsize=(120, 120))

show(saved_raster_data, transform=raster_transform, ax=ax, cmap=plt.cm.Blues)

ax.set_title("Peso de uso de suelo en relación con la distribución de la población en la Ciudad de México (rasterizado)")
ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

plt.show()